In [1]:
from model import *
from data import *

import glob
import cv2

import tensorflow as tf
from tensorflow.metrics import *

from keras.metrics import *
import keras.backend as K

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow.metrics'

In [2]:
data = './data/sky/iphone_full'
images_dir = 'val'
masks_dir = 'val_labels'

In [3]:
models_wieghts = ['./models/Unet_modify_100.27-0.97.hdf5',
                 './models/Unet_modify_100.49-0.98.hdf5',
                  './models/Unet_modify_500.29-0.99.hdf5',
                 './models/Unet_modify_500.54-0.99.hdf5',
                 './models/Unet_modify_1000.28-0.99.hdf5',
                 './models/Unet_modify_full.30-0.99.hdf5',
                 './models/Unet.104-0.99_b.hdf5']

In [4]:
metrics_names = ['Accuracy', 'F1 score', 'IoU score']
models_labels = ['Unet with ResNet 50 as encoder trained on 100 images (30 epochs).',
                 'Unet with ResNet 50 as encoder trained on 100 images (50 epochs).',
                 'Unet with ResNet 50 as encoder trained on 500 images (30 epochs).',
                'Unet with ResNet 50 as encoder trained on 500 images (50 epochs).',
                'Unet with ResNet 50 as encoder trained on 1000 images (30 epochs).',
                'Unet with ResNet 50 as encoder trained on 1263 images (30 epochs).',
                'Unet with ResNet 50 as encoder trained on 1263 images (104 epochs).']

In [5]:
def images_masks_lists(data_path, images_dir, masks_dir):
    true_masks = glob.glob(os.path.join(data_path, (masks_dir + '/*')))
    true_masks.sort()
    
    images = glob.glob(os.path.join(data_path, (images_dir + '/*')))
    images.sort()
    
    return true_masks, images

In [6]:
def iou_sc(y_true, y_pred):
    metric = mean_iou(labels=y_true, predictions=y_pred, num_classes=2)
    return metric

def f1_sc(y_true, y_pred):
    p = precision(labels=y_true, predictions=y_pred)
    r = recall(labels=y_true, predictions=y_pred)
    
    result = tf.multiply(tf.constant(2, dtype=tf.float32), tf.divide(tf.multiply(p, r), tf.add(p, r)))
    
    return result

def ac_sc(y_true, y_pred):
    metric = accuracy(labels=y_true, predictions=y_pred)
    return metric

In [7]:
def calc_metrics(model_wts, model, images, true_masks, round_func):    
    result = []
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

    lab_placeholder = tf.placeholder(np.float64, shape=(256, 256))
    pred_placeholder = tf.placeholder(np.float64, shape=(256, 256))

    metric_1 = iou_sc(lab_placeholder, pred_placeholder)
    metric_2 = f1_sc(lab_placeholder, pred_placeholder)
    metric_3 = ac_sc(lab_placeholder, pred_placeholder)

    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    for weigths_path in model_wts:        
        model.load_weights(weigths_path)
        
        ac_scores = []
        f1_scores = []
        iou_scores = []

        for i in range(len(images)):
            image = cv2.imread(images[i])
            image = cv2.cvtColor(image, code=cv2.COLOR_BGR2RGB) / 255.
            image = cv2.resize(image, (256, 256))
            image = np.expand_dims(image, 0)

            true_mask = cv2.imread(true_masks[i], 0)
            true_mask = cv2.resize(true_mask, (256, 256))
            true_mask = true_mask / 255.
            true_mask[true_mask > 0.5] = 1
            true_mask[true_mask <= 0.5] = 0

            pred_mask = model.predict(image)[0,:,:,0]

            true_mask = true_mask.astype(np.float64)
            pred_mask = round_func(pred_mask).astype(np.float64)

            ac_scores.append(sess.run([metric_3], feed_dict={lab_placeholder: true_mask, pred_placeholder: pred_mask})[0])
            f1_scores.append(sess.run([metric_2], feed_dict={lab_placeholder: true_mask, pred_placeholder: pred_mask})[0])
            iou_scores.append(sess.run([metric_1], feed_dict={lab_placeholder: true_mask, pred_placeholder: pred_mask})[0])        
        
        mean_ac_score = np.mean(np.array(ac_scores)[:, 1])
        mean_f1_score = np.mean(np.array(f1_scores)[:, 1])
        mean_iou_score = np.mean(np.array(iou_scores)[:, 0])
        
        result.append((mean_ac_score, mean_f1_score, mean_iou_score))
    
    return result

In [8]:
def print_metrics(model_label, metrics):
    print('[INFO] :: {}'.format(model_label))
    for metric in metrics:
        print('{0} : {1}'.format(metric[0], metric[1]))
    print('_________________________________')

In [9]:
def rnd(value):
    return value.round()

def threshold(value):
    
    value = cv2.normalize(value, None, 255,0, cv2.NORM_MINMAX, cv2.CV_8UC1)
    _,thresh = cv2.threshold(value, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return thresh

def threshold_and_contour(value):
    
    value = cv2.normalize(value, None, 255,0, cv2.NORM_MINMAX, cv2.CV_8UC1)
    _,thresh = cv2.threshold(value, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    _, contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    contour_list = []
    area_threshold = 500

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > area_threshold :
            contour_list.append(contour)

    countMask = np.zeros(thresh.shape, dtype='uint8')
    cv2.drawContours(countMask, contour_list, -1, (255, 255, 255), cv2.FILLED)
    
    res_mask = cv2.bitwise_not(np.add(cv2.bitwise_not(thresh), cv2.bitwise_not(countMask)))
    
    res_mask = cv2.normalize(res_mask, None, 1,0, cv2.NORM_MINMAX, cv2.CV_8UC1)
    
    return res_mask
    
    

In [10]:
msks, imgs = images_masks_lists(data_path=data, images_dir=images_dir, masks_dir=masks_dir)
model = unet_modify()

num = 0
for metrics in calc_metrics(models_wieghts, model, imgs, msks, threshold_and_contour):
    model_label = models_labels[num]
    metrics = [(metrics_names[i], metrics[i]) for i in range(3)]
    num += 1
    
    print_metrics(model_label, metrics)

/home/dmitrii/.virtenvs/FaceIidentification/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/home/dmitrii/sky_detect/Semantic-Segmentation-Suite/UNET/sky-segmentation/keras_unet/model.py:79: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


[INFO] :: Unet with ResNet 50 as encoder trained on 100 images (30 epochs).
Accuracy : 0.9803081750869751
F1 score : 0.9770065546035767
IoU score : 0.9239333868026733
_________________________________
[INFO] :: Unet with ResNet 50 as encoder trained on 100 images (50 epochs).
Accuracy : 0.9770894646644592
F1 score : 0.9748114347457886
IoU score : 0.9546597003936768
_________________________________
[INFO] :: Unet with ResNet 50 as encoder trained on 500 images (30 epochs).
Accuracy : 0.9802210330963135
F1 score : 0.978460967540741
IoU score : 0.9606500864028931
_________________________________
[INFO] :: Unet with ResNet 50 as encoder trained on 500 images (50 epochs).
Accuracy : 0.9831638336181641
F1 score : 0.9817739725112915
IoU score : 0.9665442109107971
_________________________________
[INFO] :: Unet with ResNet 50 as encoder trained on 1000 images (30 epochs).
Accuracy : 0.9850782752037048
F1 score : 0.983899712562561
IoU score : 0.9703385829925537
______________________________